In [2]:
!pip install networkx 
import networkx as nx

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
graph = nx.read_gexf("../data/concepnet-best.gexf")
dot = nx.nx_pydot.to_pydot(graph)
dot.set_rankdir("LR")
dot.write_pdf("../data/conceptnet.pdf")

In [4]:
from snn.base import Entity, Relation, SNN

Using TensorFlow backend.


In [5]:
nodes = list(nx.topological_sort(graph))
nodes.reverse()

In [55]:
entities = {}

for n in nodes:
    parents = list(graph.neighbors(n))
    parents = {entities[p] for p in parents}
    entity = Entity(n, *parents)
    entities[n] = entity
    
entities['*'] = Entity('everything', *[e for e in entities.values() if not e.children])

In [7]:
import pandas as pd
import tqdm
import json

In [58]:
snn_relations = []
relation_labels = set()

for i, (src, dst, r) in enumerate(json.load(open("../data/concepnet-relations.json"))):
    r = r.split("/")[-1]
    if r not in relation_labels:
        snn_relations.append(Relation(label=r, src=entities['*'], dst=entities['*']))
        relation_labels.add(r)
    
snn_relations

[Synonym(<everything> , <everything>),
 Causes(<everything> , <everything>),
 Antonym(<everything> , <everything>),
 Desires(<everything> , <everything>),
 UsedFor(<everything> , <everything>),
 InstanceOf(<everything> , <everything>),
 DerivedFrom(<everything> , <everything>),
 HasContext(<everything> , <everything>),
 SimilarTo(<everything> , <everything>),
 PartOf(<everything> , <everything>),
 AtLocation(<everything> , <everything>),
 MannerOf(<everything> , <everything>),
 DistinctFrom(<everything> , <everything>),
 NotDesires(<everything> , <everything>),
 EtymologicallyRelatedTo(<everything> , <everything>),
 occupation(<everything> , <everything>),
 HasPrerequisite(<everything> , <everything>),
 Entails(<everything> , <everything>),
 HasSubevent(<everything> , <everything>)]

In [12]:
def build_concepnet_graph():
    full_conceptnet = nx.read_gexf("../data/conceptnet.gexf")

    concepts = set("/c/en/%s" % n for n in nodes)
    concepts

    conceptnet_is_a = nx.DiGraph()

    for n in concepts:
        conceptnet_is_a.add_node(n)

    for node in tqdm.tqdm(full_conceptnet.nodes):
        descendants = list(nx.descendants(full_conceptnet, node))
        for des in descendants:        
            if des in concepts:
                conceptnet_is_a.add_edge(node, des)

    nx.write_gexf(conceptnet_is_a, "conceptnet-is-a.gexf")

In [10]:
conceptnet_is_a = nx.read_gexf("conceptnet-is-a.gexf")

In [11]:
def get_classes(node):
    return [entities[d.split("/")[3]] for d in nx.descendants(conceptnet_is_a, node)]

In [24]:
x = list(range(len(conceptnet_is_a.nodes)))
y = [get_classes(n) for n in tqdm.tqdm(conceptnet_is_a.nodes)]

100%|██████████| 134360/134360 [00:20<00:00, 6569.62it/s]


In [64]:
from keras.layers import Input, Embedding, Flatten

In [65]:
inp = Input(shape=(1,))
em = Embedding(input_dim=len(x), output_dim=32, input_length=1)(inp)
fl = Flatten()(em)

In [66]:
snn = SNN(entities=entities.values(), relations=snn_relations)
out = snn(fl)

In [69]:
model = snn.build(inp)

from keras.utils.vis_utils import model_to_dot

dot = model_to_dot(model)
dot.write_pdf("conceptnet-model.pdf")

In [41]:
model.fit(x,y, batch_size=512, epochs=5)

Epoch 1/5
134360/134360 [==============================] - 24s 178us/step - loss: 0.0082
Epoch 2/5
134360/134360 [==============================] - 24s 179us/step - loss: 0.0072
Epoch 3/5
134360/134360 [==============================] - 24s 181us/step - loss: 0.0049
Epoch 4/5
134360/134360 [==============================] - 24s 182us/step - loss: 0.0030
Epoch 5/5
134360/134360 [==============================] - 25s 183us/step - loss: 0.0018


In [42]:
yp = model.predict(x)

In [48]:
(y == (yp>0.5).astype(int)).astype(float).mean()

0.9996288436184239